In [1]:
%load_ext autoreload
%autoreload 2

%connect_info

{
  "shell_port": 43095,
  "iopub_port": 36343,
  "stdin_port": 59907,
  "control_port": 46375,
  "hb_port": 48417,
  "ip": "127.0.0.1",
  "key": "6a394b27-fb346cbe4508ad5987194d5d",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-616ab64a-d877-4f1f-9726-1767a15111d9.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
import django

In [3]:
django.setup()

In [4]:
"""
Project Configuration Sheets
"""
FIN_MAN = "FIN_MAN"
FIN_MAN_PROJECT_SHEET_NAME = "Finman Project Configuration"
FIN_MAN_CONFIGURATION_SHEET_NAME="FinMan Configuration_v4 Dev Loading"
VENDOR_CONFIGURATION_SHEET_NAME = "Vendor Configuration_v1 - Test"

In [5]:
"""
Populate project and project roles
"""

from django.db import transaction

@transaction.atomic()
def populate_project_and_roles():
    from ib_iam.populate.populate_utils import populate_projects
    populate_projects(spread_sheet_name=FIN_MAN_PROJECT_SHEET_NAME)

    from ib_tasks.populate.population_utils import populate_project_roles
    populate_project_roles(FIN_MAN_CONFIGURATION_SHEET_NAME)

In [6]:
"""
Populate tasks related data
"""

from django.db import transaction


@transaction.atomic()
def populate_task_and_boards_related_data():
    
    project_configuration_sheets = [
        FIN_MAN_CONFIGURATION_SHEET_NAME
    ]

    for project_configuration_sheet in project_configuration_sheets:
        configuration_sheet, project_id = project_configuration_sheet
        from ib_tasks.populate.population_utils import populate_data
        populate_data(project_id, configuration_sheet)
        
        from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

In [7]:
"""
Populate project related data
"""
from django.db import transaction

@transaction.atomic()
def populate_project_related_data():

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(FIN_MAN_PROJECT_SHEET_NAME)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(FIN_MAN_PROJECT_SHEET_NAME)

In [ ]:
populate_project_and_roles()
populate_task_and_boards_related_data()
populate_project_related_data()

In [ ]:
from ib_tasks.populate.write_stage_actions import writing_data_to_stage_actions_logic
writing_data_to_stage_actions_logic()

In [ ]:
"""
Populate Ib IAM Related Data For FinMan Project Configuration

Note:- While Running the this below script need to Run the ELASTIC SEARCH SERVER
"""
from ib_iam.populate.populate_utils import populate
populate(FIN_MAN_CONFIGURATION_SHEET_NAME)

from ib_users.models import *
UserAccount.objects.create_superuser(username='admin', password='admin@123', email='admin@example.com')
UserAccount.objects.create_superuser(username='fduser', password='fduser@123', email='fduser@example.com')

In [ ]:
"""
Populate Elastic Search Data For Cities

"""
from ib_iam.populate.populate_elastic_search_data import populate_data
populate_data()

In [ ]:
from elasticsearch_dsl import connections
from django.conf import settings
es = connections.create_connection(
hosts=[settings.ELASTICSEARCH_ENDPOINT], timeout=20)

In [ ]:
from ib_tasks.documents.elastic_task import TASK_INDEX_NAME
es.indices.create(index = TASK_INDEX_NAME)